In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from audioDataLoader import audioDataloader
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sys
import neuro
from py_apps import utils
from py_apps.utils.common_utils import read_network
from py_apps.utils.common_utils import load_json_arg
from py_apps.utils.neuro_help import *
import numpy as np

In [3]:
parser = argparse.ArgumentParser(description="Classification Application Driver")
parser.add_argument("--activity", "-a", required=True, type=str, choices=["train", "test"], help="activity to perform")
args = parser.parse_args(["--activity","train"])

In [4]:
parser = argparse.ArgumentParser(description="Classification Application Driver")
parser.add_argument("--activity", "-a", required=True, type=str, choices=["train", "test"], help="activity to perform")
parser.add_argument("--network_filename", default=None, type=str, help="location to store the best network file produced if training or network to load if testing")
parser.add_argument("--sim_time", default=50, type=float, help="the simulation timefor each data instance")
parser.add_argument("--eons_params", default="config/eons.json", type=str, help="JSON file with EONS parameters")
parser.add_argument("--extra_eons_params", default="{}", type=str ,help="JSON file or JSON string updating EONS parameters from configuration file")
parser.add_argument("--epochs", default=50, type=int, help="epochs for eons")
parser.add_argument("--max_fitness", default=1e+06, type=float, help="max fitness for eons")
parser.add_argument("--processes", default=1, type=int, help="processes for EONS")
parser.add_argument("--test_seed", default=1234, type=int, help="testing seed")

# Classification params
add_class_arguments(parser)

# Proc params 
add_proc_arguments(parser)

    # Encoder information
add_coder_arguments(parser)

# Printing params
add_printing_arguments(parser)

In [5]:
params = """
	-a train \
	--proc_params ./config/risp.json\
	--eons_params ./config/eons_SPG.json\
	--timeseries true \
	--app_type load \
	--data_np /data2/khood/GitHub/MLAudio/neuroTrain.npy \
	--labels_np /data2/khood/GitHub/MLAudio/neuroTrain_labels.npy \
	--encoder {"spikes":{"min":0,"max":1}} \
	--processes 1 \
	--sim_time 201 \
	--epochs 3 \
	--network_filename test_eons.json
    """

In [6]:
args = parser.parse_args(params.split())

In [7]:
 # Read in proc params as necessary
proc_params = load_json_arg(args.proc_params)
extra_proc_params = load_json_arg(args.extra_proc_params)
for k in extra_proc_params.keys():
    proc_params[k] = extra_proc_params[k]

# Read in EONS params
eons_params = load_json_arg(args.eons_params)

#Read in extra EONS params
extra_eons_params = load_json_arg(args.extra_eons_params)

for k in extra_eons_params.keys():
    eons_params[k] = extra_eons_params[k]
#print(eons_params)

proc_instantiation = get_proc_instantiation(args.proc_name)

# Instantiate data
X, y = setup_data(args)

config = setup_class_config(args)

In [8]:
from py_apps.utils.classify_app import ClassifyApp
import networkx as nx
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
import json

In [9]:
class ClassifyAudioApp(ClassifyApp):
    def __init__(self, config, X, y):
        super().__init__(config, X, y) 
    
    def fitness(self, net: neuro.Network, proc, id_for_printing=-1):
        
        net.prune()
        missing_path_penalty = self._count_missing_input_output_paths(net)
        if missing_path_penalty != 0:
            return missing_path_penalty
        y_predict = self.predict(proc, net, self.X_train)
        if (self.fitness_type == "accuracy"):
            ret = accuracy_score(self.y_train, y_predict) 
        elif (self.fitness_type == "f1"):
            ret = f1_score(self.y_train, y_predict, average="weighted")
        return ret
    
    def _count_missing_input_output_paths(self, net: neuro.Network):
            # Modified BFS to only visit a node when ALL of its incoming nodes have already been visited
            snnModel_raw = json.loads(json.dumps(net.as_json().to_python()))
            G = nx.Graph()

            for n in snnModel_raw['Nodes']:
                G.add_node(n['id'])

            for e in snnModel_raw['Edges']:
                G.add_edge(e['from'], e['to'], weight=round(e['values'][0],3))

            inputNodes = snnModel_raw["Inputs"]
            outputNodes = snnModel_raw["Outputs"]

            number_of_missing_paths = 0
            for subStation in inputNodes:
                for out in outputNodes:
                    if not nx.has_path(G,subStation, out):
                        number_of_missing_paths = number_of_missing_paths - 10
            return number_of_missing_paths


In [10]:
args.encoder

'{"spikes":{"min":0,"max":1}}'

In [11]:
if (args.activity == "train"):
        app = ClassifyAudioApp(config, X, y)

        train_params = {}
        train_params["eons_params"] = eons_params
        train_params["num_epochs"] = args.epochs
        train_params["num_processes"] = args.processes
        train_params["fitness"] = args.fitness
        train_start = time.time()
        app.train(train_params, proc_instantiation, proc_params)
        train_end = time.time()
        elapsed = train_end-train_start
        net = app.overall_best_net
        net.prune()
        print("Network size:", net.num_nodes(), net.num_edges())
        print("Training Time:", elapsed)

: 

## Compairing running configs

<pre>
eval "$(conda shell.bash hook)"
conda activate mlaudio
python snn.py\
	-a train \
	--proc_params ./config/risp.json\
	--eons_params ./config/eons.json\
	--timeseries true \
	--app_type load \
	--data_np /data2/khood/GitHub/MLAudio/numpyData/valid.npy \
	--labels_np /data2/khood/GitHub/MLAudio/numpyData/valid_labels.npy \
	--encoder config/encoder.json \
	--processes <strong><ins>4</ins></strong> \
	--sim_time 7201 \
	--epochs 3 \
	--network_filename test_eons.json\
    --split 0
</pre>

#### result
<pre>
Reading classification params
Reading proc params
Reading encoder information
Reading printing params
Reading in proc params as necessary
Reading in EONS params
Reading in extra EONS params
Instantiate data
Starting training
Epoch:   0     Time: 2410.7     Best: -143920    Num_Synapses: 2775
Epoch:   1     Time: 2722.6     Best: -143660    Num_Synapses: 2775
Epoch:   2     Time: 2775.8     Best: -143640    Num_Synapses: 2774
Network size: 7203 2711
Training Time: 7911.819289684296
</pre>

<pre>
eval "$(conda shell.bash hook)"
conda activate mlaudio
python snn.py\
	-a train \
	--proc_params ./config/risp.json\
	--eons_params ./config/eons.json\
	--timeseries true \
	--app_type load \
	--data_np /data2/khood/GitHub/MLAudio/numpyData/valid.npy \
	--labels_np /data2/khood/GitHub/MLAudio/numpyData/valid_labels.npy \
	--encoder config/encoder.json \
	--processes <strong><ins>16</strong></ins> \
	--sim_time 7201 \
	--epochs 3 \
	--network_filename test_eons.json\
    --split 0
</pre>

#### result

<pre>
Reading classification params
Reading proc params
Reading encoder information
Reading printing params
Reading in proc params as necessary
Reading in EONS params
Reading in extra EONS params
Instantiate data
Starting training
Epoch:   0     Time: 4390.3     Best: -143300    Num_Synapses: 2775
Epoch:   1     Time: 5330.0     Best: -143300    Num_Synapses: 2775
Epoch:   2     Time: 5183.9     Best: -143300    Num_Synapses: 2775
Network size: 7203 2708
Training Time: 14907.133779287338
</pre>

<pre>
eval "$(conda shell.bash hook)"
conda activate mlaudio
python snn.py\
	-a train \
	--proc_params ./config/risp.json\
	--eons_params ./config/eons.json\
	--timeseries true \
	--app_type load \
	--data_np /data2/khood/GitHub/MLAudio/numpyData/valid.npy \
	--labels_np /data2/khood/GitHub/MLAudio/numpyData/valid_labels.npy \
	--encoder config/encoder.json \
	--processes <strong><ins>1</strong></ins> \
	--sim_time 7201 \
	--epochs 3 \
	--network_filename test_eons.json\
    --split 0
</pre>

#### result

<pre>
Reading classification params
Reading proc params
Reading encoder information
Reading printing params
Reading in proc params as necessary
Reading in EONS params
Reading in extra EONS params
Instantiate data
Starting training
Epoch:   0     Time:  826.8     Best: -143730    Num_Synapses: 2775
Epoch:   1     Time:  865.8     Best: -143730    Num_Synapses: 2775
Epoch:   2     Time:  861.8     Best: -143730    Num_Synapses: 2775
Network size: 7203 2725
Training Time: 2557.108283996582
</pre>